In [97]:
import pandas as pd
import nltk
import numpy as np

Загрузим данные

In [98]:
data = pd.read_table('products_sentiment_train.tsv', header=None)

Сформируем список меток классов и текстов

In [99]:
labels = data[1].tolist()
data = data[0].tolist()

In [100]:
texts = [word for word in data]

Обучим классификатор

In [113]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.cross_validation import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

In [117]:
pipeline = Pipeline(steps = [('vect', CountVectorizer(max_df=0.5, ngram_range=(1, 3))), 
                              ('tfidf', TfidfTransformer(norm='l2', use_idf=True)),
                              ('clf', LogisticRegression(C=10000, penalty='l2'))])
cross_val_score(pipeline, texts, labels, cv=5).mean()

0.79248073425458909

In [124]:
def lemmatization_data(data):
    wnl = nltk.WordNetLemmatizer()
    tokens_list = [nltk.word_tokenize(raw) for raw in data]
    for tokens in tokens_list:
        tokens = [wnl.lemmatize(t) for t in tokens]
        
    data_processing = [' '.join(x) for x in tokens_list]
    return data_processing

In [139]:
pipeline = Pipeline(steps = [('vect', CountVectorizer(max_df=0.5, ngram_range=(1, 3))), 
                              ('tfidf', TfidfTransformer(norm='l2', use_idf=True)),
                              ('clf', LogisticRegression(C=10000, penalty='l2'))])
cross_val_score(pipeline, lemmatization_data(texts), labels, cv=5).mean()

0.79398449365308532

In [140]:
test = pd.read_csv('products_sentiment_test.tsv', sep='\t')
X_test = test.text


pipeline.fit(texts, labels)
y = pipeline.predict(lemmatization_data(X_test))

for_submission = pd.DataFrame(np.vstack((test.Id, y)).T, columns=['Id', 'y'])
for_submission.to_csv('test_for_submission_LinearSVC.csv', index=None)